In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import os, sys, math
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from tpot import TPOTClassifier

In [4]:
train_data = pd.read_csv('../Data/train_yaOffsB.csv')
test_data = pd.read_csv('../Data/test_pFkWwen.csv')
sample_submission = pd.read_csv('../Data/sample_submission_O1oDc4H.csv')

In [5]:
ID = 'ID'
target = 'Crop_Damage'

In [6]:
categorical_columns = ['Season', 'Pesticide_Use_Category']

# Data Preprocessing

## Missing Imputation

In [7]:
train_data['Number_Weeks_Used'] = train_data['Number_Weeks_Used'].fillna(0)
test_data['Number_Weeks_Used'] = test_data['Number_Weeks_Used'].fillna(0)

## One Hot Encoding

In [8]:
train_data['is_train'] = True
test_data['is_train'] = False

In [9]:
for column in categorical_columns:
    train_data[column] = train_data[column].apply(lambda x: f'{column}_{x}')
    test_data[column] = test_data[column].apply(lambda x: f'{column}_{x}')
    
for idx, column in enumerate(categorical_columns):
    _tmp = pd.concat([train_data[['is_train', column]], test_data[['is_train', column]]])
    _tmp_ohe = pd.get_dummies(_tmp[column])

    if idx==0:
        _tmp_final = pd.concat([_tmp[['is_train']], _tmp_ohe], axis=1)
    else:
        _tmp_final = pd.concat([_tmp_final, _tmp_ohe], axis=1)

train_data = pd.concat([train_data.drop(categorical_columns+['is_train'], axis=1), _tmp_final[_tmp_final['is_train']]], axis=1)
test_data = pd.concat([test_data.drop(categorical_columns+['is_train'], axis=1), _tmp_final[~_tmp_final['is_train']]], axis=1)

In [10]:
X_train, X_val, y_train, y_val = train_test_split(train_data.drop(['ID','Crop_Damage','is_train'], axis=1), 
                                                  train_data[['Crop_Damage']], test_size = 0.3, 
                                                  random_state = 1234, stratify = train_data['Crop_Damage'])

In [11]:
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, random_state=1234)

In [ ]:
tpot.fit(X_train, y_train.values.ravel())


Generation 1 - Current best internal CV score: 0.845096463022508
Generation 2 - Current best internal CV score: 0.8454662379421221
Generation 3 - Current best internal CV score: 0.8454662379421221